<a href="https://colab.research.google.com/github/Guilli12pm/ML_proj/blob/master/project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning project

In [4]:
!git clone https://github.com/Guilli12pm/ML_proj

Cloning into 'ML_proj'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 209 (delta 62), reused 161 (delta 24), pack-reused 0
Receiving objects: 100% (209/209), 16.66 MiB | 20.55 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [7]:
!ls ML_proj/

check_photos.py			     haarcascade_smile.xml
create_data.py			     next_ite
Database			     project_ML.ipynb
eyes_check.py			     __pycache__
haarcascade_eye_tree_eyeglasses.xml  README.md
haarcascade_eye.xml		     reset.py
haarcascade_frontalface_default.xml
